# Engine Earth

## Google Earth Engine (GEE)

In [2]:
import ee
import geemap

/opt/anaconda3/envs/gee310/lib/python3.10/site-packages/geemap/conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
# Confirm authentication
ee.Authenticate(auth_mode='notebook')
print("Authentification:" ,ee.Authenticate())

# If Authentication issue, force in CLI
    # earthengine authenticate --quiet --force

# Initialize the Earth Engine module.
google_project = "wildfire-canada-475322"
ee.Initialize(project=str(google_project))
print("Initialized:", ee.data._credentials is not None, f"with {google_project}")
print(f"Initialized With Google Project {google_project}")


# Print current EE version
print("Earth Engine Version: ",ee.__version__)

# Test data access
print("Data Access:", ee.Number(1).getInfo())


Authentification: True
Initialized: True with wildfire-canada-475322
Initialized With Google Project wildfire-canada-475322
Earth Engine Version:  0.1.419
Data Access: 1


In [12]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2020-07-01', '2020-07-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.set_center(83.277, 17.7009, 12)
m.add_layer(dataset.mean(), visualization, 'RGB')
m

Map(center=[17.7009, 83.277], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…